In [1]:
import os
import sys
import json
import time
import copy
import torch
import random
import logging
import warnings
import datetime
import torch.optim as opt

import numpy as np
import seaborn as sns
import torch.nn as nn
import matplotlib.pyplot as plt

from pathlib import Path
from torch.autograd import grad, Variable
from torch.utils.data import Subset
from data_processing import get_data_adult

warnings.filterwarnings("ignore")
device = f"cuda:5"

### Utility Functions

In [2]:
def graph(data, acc_or_l, train_or_test, lr, batch_size):
    sns.set(font_scale=1)

    plt.plot(data, 'b-', linewidth=2.0)

    plt.xlabel('Epoch')
    plt.ylabel(acc_or_l)
    plt.title(train_or_test + ' ' + acc_or_l + ' lr: ' + str(lr) + ' batch: ' + str(batch_size))
    plt.tight_layout()
    plt.show()

In [3]:
def set_random():
    random.seed(0)
    os.environ["PYTHONHASHSEED"] = str(0)
    np.random.seed(0)
    torch.manual_seed(0)
    torch.cuda.manual_seed(0)

In [4]:
def get_dataset_sample_ids_per_class(class_id, num_samples, test_loader, start_index):
    sample_list = []
    data_count = 0
    
    for i in range(len(test_loader.dataset)):
        _, y = test_loader.dataset[i]
        if class_id == y:
            data_count += 1
            if start_index < data_count <= start_index + num_samples:
                sample_list.append(i)
            elif data_count > start_index + num_samples:
                break
    
    return sample_list

In [5]:
def get_dataset_sample_ids(num_samples, test_loader, num_classes, start_index):
    sample_dict = {}
    sample_list = []
    
    for i in range(num_classes):
        sample_dict[str(i)] = get_dataset_sample_ids_per_class(i, num_samples, test_loader, start_index)
        sample_list[len(sample_list):len(sample_list)] = sample_dict[str(i)]

    return sample_dict, sample_list

In [6]:
def display_progress(text, current_step, last_step, enabled=True, fix_zero_start=True):
    if fix_zero_start:
        current_step += 1
        
    term_line_len = 80
    final_chars = [':', ';', ',','.']
    
    if text[-1:] not in final_chars:
        text = text + ' '
    
    if len(text) < term_line_len:
        bar_len = term_line_len - (len(text) + len(str(current_step)) + len(str(last_step)) + len(" / "))
    else:
        bar_len = 30

    filled_len = int(round(bar_len * current_step / float(last_step)))
    bar = '=' * filled_len + "." * (bar_len - filled_len)

    bar = f"{text}[{bar:s}] {current_step:d} / {last_step:d}"

    if current_step < last_step-1:
        sys.stdout.write("\033[K" + bar + "\r")
    else:
        sys.stdout.write(bar + "\n")

    sys.stdout.flush()

In [7]:
def save_json(json_obj, json_path, append_if_exists=False, overwrite_if_exists=True, unique_fn_if_exists=True):
    if isinstance(json_path, str):
        json_path = Path(json_path)

    if overwrite_if_exists:
        append_if_exists = False
        unique_fn_if_exists = False

    if unique_fn_if_exists:
        overwrite_if_exists = False
        append_if_exists = False
        if json_path.exists():
            time = dt.now().strftime("%Y-%m-%d-%H-%M-%S")
            json_path = json_path.parents[0] / f'{str(json_path.stem)}_{time}'\
                                               f'{str(json_path.suffix)}'

    if overwrite_if_exists:
        append_if_exists = False
        with open(json_path, 'w+') as fout:
            json.dump(json_obj, fout, indent=2)
        return

    if append_if_exists:
        if json_path.exists():
            with open(json_path, 'r') as fin:
                read_file = json.load(fin)
            read_file.update(json_obj)
            with open(json_path, 'w+') as fout:
                json.dump(read_file, fout, indent=2)
            return

    with open(json_path, 'w+') as fout:
        print('hit2')
        json.dump(json_obj, fout, indent=2)

### Influence Calculation Functions

In [8]:
def calc_point_wise(model, train_loader, test_loader, incorrect_indicies):
    test_sample_num = 1
    test_start_index = 0
    recursion_depth = 2623 
    r_averaging = 2 
    
    outdir = Path('output')
    outdir.mkdir(exist_ok=True, parents=True)
    
    influences = {}
    
    # Calculate the influence function on test sample per iteration
    for j in (incorrect_indicies):
        i = j
        
        start_time = time.time()
     
        influence, harmful, helpful, _ = calc_influence_single(model, train_loader, test_loader, i, recursion_depth, r_averaging)

        end_time = time.time()

        influences[str(i)] = {}
        _, label = test_loader.dataset[i]
        influences[str(i)]['label'] = label.cpu().numpy().tolist()
        influences[str(i)]['num_in_dataset'] = j
        influences[str(i)]['time_calc_influence_s'] = end_time - start_time
        infl = [ifl.tolist() for ifl in influence]
        influences[str(i)]['influence'] = infl
        influences[str(i)]['harmful'] = harmful[:500]
        influences[str(i)]['helpful'] = helpful[:500]

        print(f"The results for this run are:")
        print("Influences: ")
        print(influence[:5])
        print("Most harmful data IDs: ")
        print(harmful[:5])
        print("Harmful values:")
        print(influence[harmful[0]], influence[harmful[1]])
        print("Most helpful data IDs: ")
        print(helpful[:5])
        print("Helpful values: ")
        print(influence[helpful[0]], influence[helpful[1]])
        
    influence_path = outdir.joinpath(f"influence_results_{test_start_index}_{test_sample_num}.json")
    save_json(influences, influence_path)
    
    return influences
    

In [9]:
def calc_influence_single(model, train_loader, test_loader, test_id_num, recursion_depth, r):
    x_test, y_test = test_loader.dataset[test_id_num]
    x_test = test_loader.collate_fn([x_test])
    y_test = test_loader.collate_fn([y_test])
    ihvp_test_vec = calc_ihvp_single_test(model, x_test, y_test, train_loader, recursion_depth, r)
    print('Done calculating ihvp') # this gets the ihvp for one test point
    
    # calculate the influence function
    train_dataset_size = len(train_loader.dataset)
    influences = []
    
    for i in range(train_dataset_size):
        x, y = train_loader.dataset[i]
        x = train_loader.collate_fn([x])
        y = train_loader.collate_fn([y])
        
        grad_x_vec = grad_x(x, y, model) # get grad of training point
        
        tmp_influence = -sum([torch.sum(k*j).data for k, j in zip(grad_x_vec, ihvp_test_vec)]) / train_dataset_size
        
        influences.append(tmp_influence.cpu().numpy())
        
        display_progress("Calc. influence function: ", i, train_dataset_size)
        
    harmful = np.argsort(influences)
    helpful = harmful[::-1]
    
    return influences, harmful.tolist(), helpful.tolist(), test_id_num
    
    

In [10]:
def calc_ihvp_single_test(model, x_test, y_test, train_loader, recursion_depth, r):
    damp = 0
    scale = 500
   
    ihvp_test_vec_list = []
   
    print('x_test len', len(x_test))

    avg_ihvp_test_vec = ihvp_test(x_test, y_test, model, train_loader, damp, scale, recursion_depth)
                      
    for i in range(1,r):
        ith_ihvp_test_vec = ihvp_test(x_test, y_test, model, train_loader, damp, scale, recursion_depth)
        avg_ihvp_test_vec = [avg + ith for avg, ith in zip(avg_ihvp_test_vec, ith_ihvp_test_vec)]        
        display_progress("Averaging r-times: ", i, r)
       
    avg_ihvp_test_vec = [i / r for i in avg_ihvp_test_vec]
       
    return avg_ihvp_test_vec

In [11]:
#done
def ihvp_test(x_test, y_test, model, train_loader, damp, scale, recursion_depth):
    v = grad_x(x_test, y_test, model) # get grad of test point
    h_estimate = v.copy()  # h_estimate is a copy of the test pnt grad
    
    criterion = torch.nn.BCELoss()
    params = [p for p in model.parameters() if p.requires_grad]
    
    for i in range(recursion_depth):
        x, y = next(iter(train_loader))
        x, y = x.type(torch.FloatTensor).to(device), y.type(torch.FloatTensor).to(device)
        
        pred = model(x)
        loss = criterion(torch.squeeze(pred), y)
        
        hv = hvp(loss, params, h_estimate) # Hessian vector product
    
        h_estimate = [v_ + (1-damp) * h_e_ - hv_ / scale for v_, h_e_, hv_ in zip(v, h_estimate, hv)]
        
    return h_estimate

In [12]:
def grad_x(x, y, model):
    model.eval()
    criterion = torch.nn.BCELoss()
    
    x, y = x.type(torch.FloatTensor).to(device), y.type(torch.FloatTensor).to(device)
    
    pred = model(x)
    loss = criterion(pred.ravel(), y)
    
    params = [p for p in model.parameters() if p.requires_grad]
    grad_x_ = list(grad(loss, params, create_graph=True))
    
    return grad_x_

In [13]:
def hvp(loss, w, v):
    first_grads = grad(loss, w, create_graph=True)#, retain_graph=True, create_graph=True)
    elementwise_products = [torch.mul(grad_elem, v_elem.detach()) for grad_elem, v_elem in zip(first_grads, v) if grad_elem is not torch.isnan(grad_elem)]
    loss.backward(retain_graph=True, create_graph=True)
    
    print(elementwise_products[0].grad)
    #elem_prod_v = Variable(torch.FloatTensor(elementwise_products), requires_grad=True)
    
    grads_with_none = grad(elementwise_products, w, grad_outputs=[x for x in elementwise_products[0].grad])
    
    return_grads = [grad_element if grad_element is not torch.isnan(grad_element) else torch.zeros_like(x) for x, grad_element in zip(w, grads_with_none)]
    
    return grads_with_none 

In [14]:
def hvp_old(loss, w, v):
    first_grads = grad(loss, w, retain_graph=True, create_graph=True)
    #loss.backward(retain_graph=True, create_graph=True)
    
    elementwise_products = torch.zeros(1).to(device)
    
    for grad_elem, v_elem in zip(first_grads, v):
        elementwise_products += torch.sum(grad_elem * v_elem.detach())
        
    elementwise_products.backward(create_graph=True)
    #torch.nn.utils.clip_grad_norm_(w, max_norm=500.0, norm_type=2)
    return_grads = [ww.grad for ww in w]
    
    return return_grads

### Models

In [15]:
class LogReg(torch.nn.Module):
    def __init__(self, input_size):
        super(LogReg, self).__init__()
        
        self.input_size = input_size
        self.fc1 = nn.Linear(self.input_size, 1)
        
    def forward(self, x):
        prediction = torch.sigmoid(self.fc1(x))
        
        return prediction

### Training and Testing Functions

In [16]:
def train(model, train_dataloader, test_dataloader, optimizer, criterion, P, device, save=False):
    all_acc, all_loss = [], []
    correct_indx = None
    
    for i in range(30):
        model.train()

        for batch_idx, (data, target) in enumerate(train_dataloader):
            optimizer.zero_grad()

            data, target = data.type(torch.FloatTensor).to(device), target.type(torch.FloatTensor).to(device)
            pred = model(data)
            loss = criterion(torch.squeeze(pred), target)
            loss.backward()
            optimizer.step()
        
        if save:
            acc, lo, correct_indx = evaluate(model, test_dataloader, i)
            all_acc.append(acc)
            all_loss.append(lo)
            
    if save:
        torch.save(model.state_dict(), 'models/logreg.pt')
        
    return all_acc, all_loss, correct_indx

In [17]:
def evaluate(model, test_dataloader, i):
    model.eval()
    criterion = torch.nn.BCELoss()
    running_correct = 0
    total = 0
    running_loss = 0
    iters = 0
    
    correct_indx = []
    
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_dataloader):
            data, target = data.type(torch.FloatTensor).to(device), target.type(torch.FloatTensor).to(device)
            pred = model(data)
            running_loss += criterion(torch.squeeze(pred), target)
            pred_thresh = torch.squeeze(pred).round().detach().cpu().numpy()
            running_correct += np.sum(pred_thresh == target.detach().cpu().numpy())
            
            correct_indx.extend(pred_thresh == target.detach().cpu().numpy())
            
            total += len(target)
            iters = batch_idx

    acc = running_correct / total
    loss = (running_loss / iters).cpu()

    print(f'Epoch: {i} | Acc: {acc:.3f} | Loss: {loss:.3f}')

    return acc, loss, correct_indx

In [18]:
def get_test_losses(model, test_dataloader, test_id_nums):
    model.eval()
    criterion = torch.nn.BCELoss()

    actual_losses = []

    for i in test_id_nums:
        x_test, y_test = test_dataloader.dataset[i]
        x_test = test_dataloader.collate_fn([x_test])

        y_test = test_dataloader.collate_fn([y_test])
        x_test, y_test = x_test.type(torch.FloatTensor).to(device), y_test.type(torch.FloatTensor).to(device)
        outputs = model(x_test)
        actual_losses.append(criterion(outputs, y_test.unsqueeze(1)).item())

    return actual_losses

### Get Data

In [19]:
train_dataloader, test_dataloader, num_features, train_dataset, probability = get_data_adult(batch_size=256, randomize='false', epsilon=0.2)

### Train base model and get Original Losses

In [20]:
set_random()

model = LogReg(input_size=num_features)
torch.save(model.state_dict(), 'models/initial_config.pt')

optimizer = torch.optim.SGD(model.parameters(), lr=.001, weight_decay=.0001, momentum=.9, nesterov=False)
criterion = torch.nn.BCELoss()

model.to(device)
all_acc, all_loss, correct_indx = train(model, train_dataloader, test_dataloader, optimizer, criterion, None, device, save=True)

Epoch: 0 | Acc: 0.773 | Loss: 0.548
Epoch: 1 | Acc: 0.777 | Loss: 0.478
Epoch: 2 | Acc: 0.792 | Loss: 0.452
Epoch: 3 | Acc: 0.808 | Loss: 0.440
Epoch: 4 | Acc: 0.809 | Loss: 0.425
Epoch: 5 | Acc: 0.806 | Loss: 0.417
Epoch: 6 | Acc: 0.818 | Loss: 0.411
Epoch: 7 | Acc: 0.813 | Loss: 0.405
Epoch: 8 | Acc: 0.823 | Loss: 0.403
Epoch: 9 | Acc: 0.820 | Loss: 0.398
Epoch: 10 | Acc: 0.824 | Loss: 0.401
Epoch: 11 | Acc: 0.825 | Loss: 0.396
Epoch: 12 | Acc: 0.824 | Loss: 0.391
Epoch: 13 | Acc: 0.825 | Loss: 0.390
Epoch: 14 | Acc: 0.817 | Loss: 0.390
Epoch: 15 | Acc: 0.824 | Loss: 0.394
Epoch: 16 | Acc: 0.827 | Loss: 0.384
Epoch: 17 | Acc: 0.826 | Loss: 0.382
Epoch: 18 | Acc: 0.824 | Loss: 0.382
Epoch: 19 | Acc: 0.826 | Loss: 0.380
Epoch: 20 | Acc: 0.827 | Loss: 0.379
Epoch: 21 | Acc: 0.828 | Loss: 0.381
Epoch: 22 | Acc: 0.827 | Loss: 0.377
Epoch: 23 | Acc: 0.828 | Loss: 0.376


KeyboardInterrupt: 

In [ ]:
incorrect_indicies = [i for i, a in enumerate(correct_indx) if a == False][:2]

actual_losses = get_test_losses(model, test_dataloader, incorrect_indicies)

print('Original losses: ')
count = 0
for i in incorrect_indicies:
    print(f'Test Sample {i}: {actual_losses[count]}')
    count += 1
    
graph(all_loss, 'Loss', 'Testing', .001, 256)
graph(all_acc, 'Accuracy', 'Testing', .001, 256)

### Perform Influence Calculation on Test Loss

In [ ]:
influences = calc_point_wise(model, train_dataloader, test_dataloader, incorrect_indicies)

### Leave One Out Training

In [ ]:
loo_losses = {}

for i, s in enumerate(incorrect_indicies):
    indicies_to_pop = influences[str(s)]['harmful'][:50]
    print(f'\n-- LOO for Test Sample {s}')
    key = f'sample_{s}'
    loo_losses[key] = []

    for j in range(len(indicies_to_pop)):
        temp_indicies = [x for x in range(len(train_dataset))]
        temp_pop = indicies_to_pop.copy()
        index = temp_pop.pop(j)
        temp_indicies.pop(index)
        train_minus_one = Subset(train_dataset, temp_indicies)

        random.seed(0)
        os.environ["PYTHONHASHSEED"] = str(0)
        np.random.seed(0)
        torch.manual_seed(0)
        torch.cuda.manual_seed(0)

        train_dataloader = torch.utils.data.DataLoader(train_minus_one, batch_size=256, shuffle=False,
                                                       pin_memory=True, num_workers=0)
        model = LogReg(input_size=num_features)
        model.load_state_dict(torch.load('models/initial_config.pt'))

        optimizer = torch.optim.SGD(model.parameters(), lr=.001, weight_decay=.0001, momentum=.9, nesterov=False)
        criterion = torch.nn.BCELoss()
        model.to(device)

        _, _, _ = train(model, train_dataloader, test_dataloader, optimizer, criterion, None, device, save=False)

        loo_loss = get_test_losses(model, test_dataloader, [s])
        loo_losses[key].append(loo_loss[0] - actual_losses[i]) # differences

        print('-- data item: ', j, ' | ', loo_losses[key][-1])

### Graph

In [ ]:
for i, s in enumerate(incorrect_indicies):
    sns.set(font_scale=1)
    plt.scatter(loo_losses[f'sample_{s}'], [influences[str(s)]['influence'][v] for v in influences[str(s)]['harmful'][:50]])
    z = np.polyfit(loo_losses[f'sample_{s}'], [influences[str(s)]['influence'][v] for v in influences[str(s)]['harmful'][:50]],1)
    p = np.poly1d(z)
    plt.plot(loo_losses[f'sample_{s}'], p(loo_losses[f'sample_{s}']))
    plt.xlabel('Actual Loss Difference')
    plt.ylabel('Estimated Loss Difference')
    plt.title(f'Actual Loss vs. Estimated Loss Sample {s}')
    plt.tight_layout()
    plt.show()
    plt.savefig(f'sample_{i}.png')